In [2]:
# Installing the libraries for project
%%capture
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

In [3]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import shutil

Processing dataset of text documents with associated entity annotations. It uses spaCy to create a blank English language model and then converts each text into a spaCy document, while also aligning the entity annotations with the text.

In [4]:
def make_v3_dataset(data, db = []):

    nlp = spacy.blank('en') ## Create a blank English language model using spaCy
    failed_record = [] ## Initialize a list to store failed records that couldn't be processed

    ## If no existing DocBin is provided, create a new one
    if not db:
        db = DocBin()

    # Loop through each text and its annotations in the data
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text) # # Convert the raw text into a spaCy document
        ents = [] # Initialize an empty list to store the extracted entities

        # Loop through each entity annotation in the current text
        for start, end, label in annot['entities']:

           # Create a spaCy span object for each entity and try to align it with the raw text
            span = doc.char_span(start, end, label = label, alignment_mode = 'contract')

            # Check if the span was successfully created, and append it to the list of entities
            if span is None:
                print(f'empty entity, {text}, {annot["entities"]}')
            else:
                ents.append(span)

        # Try to set the entities of the document to the extracted entities
        # If this fails, add the text and its annotations to the failed_record list
        try:
            doc.ents = ents
        except:
            failed_record.append((text, annot))

        # Add the processed document to the DocBin
        db.add(doc)
    return db, failed_record  # Return the populated DocBin and the list of failed records

In [4]:
##path to annotated resume data and list of it

tagged_data = '/content/drive/MyDrive/Dissertation/tagged_data'
list_tagged_files = os.listdir(tagged_data)

In [5]:
len(list_tagged_files)

119

In [6]:
##path for storing resumes in spacy format

output_tagged_spacy_form ='/content/drive/MyDrive/Dissertation/output_tagged_spacy_form'

In [31]:
# Converting json annotated files to spacy format
for i in tqdm(range(len(list_tagged_files))):
  file_path = os.path.join(tagged_data,list_tagged_files[i])
  # print(file_path)
  with open(file_path,'r') as f:
    data = json.load(f)
  saving_path = os.path.join(output_tagged_spacy_form,list_tagged_files[i][:-5]+'.spacy')
  a,b = make_v3_dataset(data['annotations'])
  a.to_disk(saving_path)

100%|██████████| 119/119 [00:24<00:00,  4.86it/s]


Dividing annotated data in spacy format into training and validation sets.

In [24]:
# Define the source directory where your samples are located
source_directory = "/content/drive/MyDrive/Dissertation/output_tagged_spacy_form"  # Adjust the path accordingly

# Define the destination directories for training and validation sets
training_directory = "/content/drive/MyDrive/Dissertation/training_data"  # Adjust the path accordingly
validation_directory = "/content/drive/MyDrive/Dissertation/validation_data"  # Adjust the path accordingly

# Create the destination directories if they don't exist
os.makedirs(training_directory, exist_ok=True)
os.makedirs(validation_directory, exist_ok=True)

# Get a list of files in the source directory
files = os.listdir(source_directory)

# Sort the files to ensure consistent splitting
files.sort()

# Separate files into training and validation sets
for idx, filename in enumerate(files, 1):
    # If the file index is a multiple of 6, move it to the validation set
    if idx % 6 == 0:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(validation_directory, filename)
    # Otherwise, move it to the training set
    else:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(training_directory, filename)

    # Move the file to the appropriate directory
    shutil.move(source_path, destination_path)


In [9]:
# Initialization setting
!python -m spacy init fill-config /content/drive/MyDrive/Dissertation/base_config.cfg /content/drive/MyDrive/Dissertation/config.cfg


2023-07-23 21:46:14.473245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Dissertation/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
!python -m spacy train -g 0 /content/drive/MyDrive/Dissertation/config.cfg  --output /content/drive/MyDrive/Dissertation/generated_model --paths.train /content/drive/MyDrive/Dissertation/output_tagged_spacy_form --paths.dev /content/drive/MyDrive/Dissertation/validation_data

2023-07-24 09:24:20.791725: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/Dissertation/generated_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-07-24 09:24:24,964] [INFO] Set up nlp object from config
[2023-07-24 09:24:24,980] [INFO] Pipeline: ['transformer', 'ner']
[2023-07-24 09:24:24,984] [INFO] Created vocabulary
[2023-07-24 09:24:24,984] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- Th

In [13]:
!python3 -m spacy evaluate /content/drive/MyDrive/Dissertation/generated_model/model-best /content/drive/MyDrive/Dissertation/output_tagged_spacy_form --gpu-id 0 -o output_metrics

2023-07-24 11:02:27.718072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   93.87 
NER R   95.98 
NER F   94.91 
SPEED   7181  


=============================== NER (per type) ===============================

                 P       R       F
JOB TITLE    98.32   99.15   98.73
SKILL        93.30   97.06   95.14
EXPERIENCE   96.44   95.72   96.07
EDUCATION    94.56   88.96   91.67

✔ Saved results to output_metrics


In [14]:
!python3 -m spacy evaluate /content/drive/MyDrive/Dissertation/generated_model/model-best /content/drive/MyDrive/Dissertation/validation_data --gpu-id 0 -o output_metrics

2023-07-24 11:04:46.224574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   95.18 
NER R   97.16 
NER F   96.16 
SPEED   4537  


=============================== NER (per type) ===============================

                  P        R        F
JOB TITLE    100.00   100.00   100.00
EXPERIENCE   100.00   100.00   100.00
EDUCATION     94.64    85.48    89.83
SKILL         94.58    98.84    96.66

✔ Saved results to output_metrics


In [15]:
nlp = spacy.load('/content/drive/MyDrive/Dissertation/generated_model/model-best')

In [16]:
# Test example
main_str = ['''MASTER DATA MANAGER
Experience
Master Data Manager

,
01/2019

to
Current

Company Name

â€“
City

,
State

Aligning 100+ supply chain-related master data attributes to ensure the following guidelines are met:.
Data Stewardship - Appropriate business units and ownership established to validate appropriate values, definitions and
impact
documentation for all master data fields.
Data Quality - Applicable controls are set in conjecture with IT to ensure that a proactive approach is taken to new
master data.
Additionally, RPA process planned for Phase 2 to automate structured data entry and maintenance.
Data Integrity/Data Warehousing - Structure business logic to ensure BOMs and other data relationships are both
accessible and built as an
asset for the business.
Additionally, a Data Sync SOP was established for GS1 standardization.
Reporting - Automate data quality reporting for executive staff to ensure 99.8% compliance is adhered to throughout
the organization.
Additionally, provide ad-hoc tools for the business to leverage clean data cubes.
Data Governance Committee - Oversee executive strategy in institutional master data to commit to trusted, accurate, structured
and
meaningful master data.
Engaging applicable stakeholders to promote data governance, data architect, data stewardship and
data security the data governance
committee oversees the framework for complex supply chain manufacturing operations.
Director of Information Technology

,
01/2017

to
01/2019

Company Name

â€“
City

,
State

Supply Chain Management - Warehouse, 3PL, Food Services, Served as the corporate Project Manager, overseeing all implementations
(WMS, ERP, QMS, CMMS, infrastructure).
Responsibilities included progress reporting, budget control, task delegation, plan creation, vendor coordination, etc.
Developed a range of documents and performed project selection tasks such as workflow process maps, system integration
documents,
infrastructure topology, organizational policies, GDPR, UAT, gap analysis, best fit models and user training manuals.
Deployed and managed BI products to leverage company-wide data for decisionmakers, proactive analysis, and real-time resource
planning.
Utilizing SSRS/PowerBI to analyze sets of data across multiple databases, automate value-add reporting to 3PL clients,
visualize KPIs thru
the organization, create workflows for document automation and create alerts for various business units.
Launched a high-performance data warehouses and virtual machines to store, integrate, analyze, and report on enterprise data.
An
array of SSAS cubes were deployed to allow for end-user ad-hoc tabular reports.
Additionally, a complex network of interfaces
between 6+ enterprise systems, 4 SQL databases, 1 Oracle database, and other enterprise
systems (file server, exchange, EDI, etc)
was implemented to support the ETL process.
Executing infrastructure refresh for a 200,000 sqft industrial/production facility.
The deployment includes an array of 15 switches,
1,000 cable runs, racking, 100 AP, 300 cameras, NVRs, controllers, and creating floor
plans and topology documents.
Controlled a $2MM IT budget, short-term and long-term strategies, and CapEx justification across the business.
Including selecting
and negotiating with vendors, promoting operational process improvement, instituting best practices, and developing IT
roadmaps
and strategic plans.
On track to reduce IT spend by $250K by renegotiating vendor contracts, consolidating cloud services,
implementing automation tools and
upgrading legacy systems.
Responsible for a team of IT professionals and vendors assigned to all corporate and local office systems, including ERP, WMS,
logistics
systems, data centers, servers and storage, PCs, mobile devices, BDR, telecom, and help desk.
Reduced IT service desk's
response time and resolution time while increasing knowledge base and reducing reoccurring tickets by over
50%.
Information Technology Governance & Steering Committee (ITGSC) - Reporting directly to the board of directors, the
ITGSC provided
strategic direction in aligning IT projects to shareholder interest and long-term targets.
While overseeing the
committee the cross-functional team guided 12 business units across operations, sales, and manufacturing.
Additionally, the ITGSC
aided in risk management, business continuity, and organizational SOPs.
NatJan Solutions (Facility Management - Retail, Medical, Grocery Sectors.
Information Technology and Analytics Manager

,
01/2014

to
01/2017

Implemented process within external ERP portal to monetize 1000's of vendors to generate lean revenue based on financial and
operational
data analysis.
This program is estimated to bring in $325,000 in its first year, an 8% increase in margin.
Created ad-hoc and scheduled BI (PowerBI/SSRS) reports to visualize critical client metrics and reports, internal performance
KPIs,
operational resource planning, asset and revenue forecasting, P/L summaries, expense budgets, etc.
Spearheaded national RFP bids of multi-million-dollar procurement deals with several Fortune 500 retail, medical, and education
companies.
Tasks included sales analysis, margin analysis, multivariable pricing structures, vendor acquisition, developing process
flow and establishing
KPI reporting for clientele and internal use.
Designed ad-hoc SQL views, stored proceduresand reports to address company-wide data analysis and modeling.Utilized advanced VBA programming and automation software to manage two billing systems, saving an estimated $75,000 in Salary
Expenses in 2015, translating to a 40% reduction in departmental spending.
Reported on a wide array of KPIs on operations, sales, accounting, and vendors via dashboard-style (BI/SSRS) reports for COO,
CFO,
Vice President of Sales and Account Manager use.
Overhauled all company technology from on-premise systems to an array of cloud solutions, including replacing hardware and
migrating
servers and databases, this renovation saves ~$108,000 per year and provides long-term scalability.
01/2015

to
01/2017

Company Name

â€“
City

,
State

Co-Founded indoor, hydroponic farming company in South Jersey.
Bringing a hyper-local product to the Camden and Philadelphia
region HGF generated $100K in its first year via a strategic relationship with
the premier food management service in Philadelphia,
local restaurants, farmer's markets, and Whole Foods retail chain.
Sales & Operations Planning (S&OP) - The S&OP team managed bi-directional interactions between sales, marketing,
production, and
finance.
S&OP committee oversaw finished goods inventory controls, marketing vs financial targets, raw material
and supplier management, logistics
and human capital planning.
Additionally, benchmarks for product mix, SKU management, and
forecast vs demand vs actual alignment.
Operations - Oversaw the general operations of the farm, including assigning day-to-day jobs, tracking yield results, establishing
daily work
schedules, and tracking transplanting/harvesting schedules.
Automation Systems - Implemented an array of sensors to control environmental and nutrient levels.
Utilizing this data and
automation systems we drove automated pH balancing, nutrient leveling, temperature control, ventilation systems, and
established
a strong exception-based manufacturing operation.
Finance - Oversaw all aspects of the financial responsibilities including establishing a point of sales systems, P/L reports, budgeting
for OpEx
and CapEx schedules, coordinating with lenders, insurance institutions and annual tax reports.
SOPs - Instituted standard operating procedures to ensure quality standards are maintained and product consistency is maintained.
Work History
Master Data Manager

,
01/2019

to
Current

Company Name

â€“
City

,
State

Aligning 100+ supply chain-related master data attributes to ensure the following guidelines are met:.
Data Stewardship - Appropriate business units and ownership established to validate appropriate values, definitions and
impact
documentation for all master data fields.
Data Quality - Applicable controls are set in conjecture with IT to ensure that a proactive approach is taken to new
master data.
Additionally, RPA process planned for Phase 2 to automate structured data entry and maintenance.
Data Integrity/Data Warehousing - Structure business logic to ensure BOMs and other data relationships are both
accessible and built as an
asset for the business.
Additionally, a Data Sync SOP was established for GS1 standardization.
Reporting - Automate data quality reporting for executive staff to ensure 99.8% compliance is adhered to throughout
the organization.
Additionally, provide ad-hoc tools for the business to leverage clean data cubes.
Data Governance Committee - Oversee executive strategy in institutional master data to commit to trusted, accurate, structured
and
meaningful master data.
Engaging applicable stakeholders to promote data governance, data architect, data stewardship and
data security the data governance
committee oversees the framework for complex supply chain manufacturing operations.
Director of Information Technology

,
01/2017

to
01/2019

Company Name

â€“
City

,
State

Supply Chain Management - Warehouse, 3PL, Food Services, Served as the corporate Project Manager, overseeing all implementations
(WMS, ERP, QMS, CMMS, infrastructure).
Responsibilities included progress reporting, budget control, task delegation, plan creation, vendor coordination, etc.
Developed a range of documents and performed project selection tasks such as workflow process maps, system integration
documents,
infrastructure topology, organizational policies, GDPR, UAT, gap analysis, best fit models and user training manuals.
Deployed and managed BI products to leverage company-wide data for decisionmakers, proactive analysis, and real-time resource
planning.
Utilizing SSRS/PowerBI to analyze sets of data across multiple databases, automate value-add reporting to 3PL clients,
visualize KPIs thru
the organization, create workflows for document automation and create alerts for various business units.
Launched a high-performance data warehouses and virtual machines to store, integrate, analyze, and report on enterprise data.
An
array of SSAS cubes were deployed to allow for end-user ad-hoc tabular reports.
Additionally, a complex network of interfaces
between 6+ enterprise systems, 4 SQL databases, 1 Oracle database, and other enterprise
systems (file server, exchange, EDI, etc)
was implemented to support the ETL process.
Executing infrastructure refresh for a 200,000 sqft industrial/production facility.
The deployment includes an array of 15 switches,
1,000 cable runs, racking, 100 AP, 300 cameras, NVRs, controllers, and creating floor
plans and topology documents.
Controlled a $2MM IT budget, short-term and long-term strategies, and CapEx justification across the business.Including selecting
and negotiating with vendors, promoting operational process improvement, instituting best practices, and developing IT
roadmaps
and strategic plans.
On track to reduce IT spend by $250K by renegotiating vendor contracts, consolidating cloud services,
implementing automation tools and
upgrading legacy systems.
Responsible for a team of IT professionals and vendors assigned to all corporate and local office systems, including ERP, WMS,
logistics
systems, data centers, servers and storage, PCs, mobile devices, BDR, telecom, and help desk.
Reduced IT service desk's
response time and resolution time while increasing knowledge base and reducing reoccurring tickets by over
50%.
Information Technology Governance & Steering Committee (ITGSC) - Reporting directly to the board of directors, the
ITGSC provided
strategic direction in aligning IT projects to shareholder interest and long-term targets.
While overseeing the
committee the cross-functional team guided 12 business units across operations, sales, and manufacturing.
Additionally, the ITGSC
aided in risk management, business continuity, and organizational SOPs.
NatJan Solutions (Facility Management - Retail, Medical, Grocery Sectors.
Information Technology and Analytics Manager

,
01/2014

to
01/2017

Implemented process within external ERP portal to monetize 1000's of vendors to generate lean revenue based on financial and
operational
data analysis.
This program is estimated to bring in $325,000 in its first year, an 8% increase in margin.
Created ad-hoc and scheduled BI (PowerBI/SSRS) reports to visualize critical client metrics and reports, internal performance
KPIs,
operational resource planning, asset and revenue forecasting, P/L summaries, expense budgets, etc.
Spearheaded national RFP bids of multi-million-dollar procurement deals with several Fortune 500 retail, medical, and education
companies.
Tasks included sales analysis, margin analysis, multivariable pricing structures, vendor acquisition, developing process
flow and establishing
KPI reporting for clientele and internal use.
Designed ad-hoc SQL views, stored procedures and reports to address company-wide data analysis and modeling.
Utilized advanced VBA programming and automation software to manage two billing systems, saving an estimated $75,000 in Salary
Expenses in 2015, translating to a 40% reduction in departmental spending.
Reported on a wide array of KPIs on operations, sales, accounting, and vendors via dashboard-style (BI/SSRS) reports for COO,
CFO,
Vice President of Sales and Account Manager use.
Overhauled all company technology from on-premise systems to an array of cloud solutions, including replacing hardware and
migrating
servers and databases, this renovation saves ~$108,000 per year and provides long-term scalability.
01/2015

to
01/2017

Company Name

â€“
City

,
State

Co-Founded indoor, hydroponic farming company in South Jersey.
Bringing a hyper-local product to the Camden and Philadelphia
region HGF generated $100K in its first year via a strategic relationship with
the premier food management service in Philadelphia,
local restaurants, farmer's markets, and Whole Foods retail chain.
Sales & Operations Planning (S&OP) - The S&OP team managed bi-directional interactions between sales, marketing,
production, and
finance.
S&OP committee oversaw finished goods inventory controls, marketing vs financial targets, raw material
and supplier management, logistics
and human capital planning.
Additionally, benchmarks for product mix, SKU management, and
forecast vs demand vs actual alignment.
Operations - Oversaw the general operations of the farm, including assigning day-to-day jobs, tracking yield results, establishing
daily work
schedules, and tracking transplanting/harvesting schedules.
Automation Systems - Implemented an array of sensors to control environmental and nutrient levels.
Utilizing this data and
automation systems we drove automated pH balancing, nutrient leveling, temperature control, ventilation systems, and
established
a strong exception-based manufacturing operation.
Finance - Oversawall aspects of the financial responsibilities including establishing a point of sales systems, P/L reports, budgeting
for OpEx
and CapEx schedules, coordinating with lenders, insurance institutions and annual tax reports.
SOPs - Instituted standard operating procedures to ensure quality standards are maintained and product consistency is maintained.
Education
Bachelor of Science

:
Management of Information Systems

,
2017

Associates of Science

:
Accounting

,
2015

Rowan College

Pursuing Project Management Professional (PMP) certification

Rowan University

Summary
Zeel Patel is an Information Technology Manager with a focus on IT transformation and process improvement currently looking for opportunities in
the greater Miami, FL metropolitan area. Zeel has experience working with corporate data in the areas of data presentation, structured and
unsecured datasets, data mining, statistical analysis, UI creation, data warehouse management, and assessing data quality. Additionally, Zeel's
diverse background has allowed him to succeed in a project management and IT management role. In this role, Zeel has engaged in holistic
business analysis, process improvement, strategic planning, budgeting, contract negotiations, vendor relations, resource management, instituting
best practices, and overseeing/implementing multiple enterprise systems (ERP, WMS, CRM, WOM, QMS, SCP).
Highlights
Automation tools, HTML, RetailMicrosoft Office Suite with deep Office 365, Dynamics 365, Project, PowerApps, PowerBI,
Flow,
Sharepoint, Excel, Access, Intune, Visio, EMS, and Azure proficiency
Programming/Databases: SQL (SSRS, SSIS, SSAS), VBA, DAX, HTML, CSS, VBA,
VB.NET, R, Powershell, Python, Oracle
Business Intelligence Packages: PowerBI, Qlik (QlikView/QlikSense), SiSense, Tableau,
Datorama, Yellowfin, Crystal, SSRS
Strong knowledge of AWS, Azure, Cisco Switch Management, VMWare, Hyper-V, RDP,
Automation Anywhere, Active Directory,
And hardware and software administration for iOS, Android, Windows, Red Hat Linux, RF
devices
VB.NET, EDI, Procurement, Ventilation
Accounting, ERP, Programming, Visio
Active Directory, ETL, Progress, VBA
Premier, Facility Management, Project Management, Workflow
Ad, Finance, Python
AP, Financial, Quality
Approach, Floor plans, Real-time
Architect, Forecasting, Red Hat Linux
Automate, Functional, Renovation
Automation, Help desk, Reporting
Automation tools, HTML, Retail
Billing systems, Information
Technology, RFP
Budgeting, Insurance, Risk
management
Budgets, Inventory, Sales
Budget, Logic, Sales analysis
BI, Logistics, Servers
Business Intelligence, Marketing,
SOP
Cable, Access, SQL
Cisco, Excel, Strategy
Hardware, Exchange, Strategic
Contracts, Microsoft Office Suite,
Strategic plans
Crystal, Office, Structured
CSS, Sharepoint, Supplier
management
Clientele, Windows, Supply chain
Client, Negotiating, Supply Chain
Management
Clients, Enterprise, Switches
Data analysis, Network, Switch
Data entry, Oracle, System
integration
Data analysis and modeling, Oracle
database, Tableau
Data warehouses, Organizational,
Tax
Databases, Cameras, User training
Data Warehousing, Policies,
Telecom
Direction, Pricing, Translating
Documentation, Process
improvement, Upgrading
Skills
Microsoft Office Suite with deep Office 365, Dynamics 365, Project, PowerApps, PowerBI, Flow,
SSharepoint, Excel, Access, Intune, Visio, EMS, and Azure proficiency
PProgramming/Databases: SQL (SSRS, SSIS, SSAS), VBA, DAX, HTML, CSS, VBA, VB.NET, R, Powershell, Python, Oracle
BBusiness Intelligence Packages: PowerBI, Qlik (QlikView/QlikSense), SiSense, Tableau, Datorama, Yellowfin, Crystal, SSRS
SStrong knowledge of AWS, Azure, Cisco Switch Management, VMWare, Hyper-V, RDP, Automation Anywhere, Active Directory,
Aand hardware and software administration for iOS, Android, Windows, Red Hat Linux, RF devices,
VB.NET, accounting, Active Directory, premier, ad, AP, approach, architect, Automate, Automation, automation tools, billing systems,
budgeting, budgets, budget, BI, Business Intelligence, cable, Cisco, hardware, contracts, Crystal, CSS, clientele, client, clients, data analysis, data
entry, data analysis and modeling, data warehouses, Databases, Data Warehousing, direction, documentation, EDI, ERP, ETL, Facility
Management, Finance, financial, floor plans, forecasting, functional, help desk, HTML, Information Technology, insurance, inventory, logic,
logistics, marketing, Access, Excel, exchange, Microsoft Office Suite, Office, Sharepoint, Windows, negotiating, enterprise, network, Oracle,
Oracle database, organizational, cameras, policies, pricing, process improvement, procurement, Programming, progress, Project Management,
Python, Quality, real-time, Red Hat Linux, renovation, reporting, Retail, RFP, risk management, Sales, sales analysis, servers, SOP, SQL,
strategy, strategic, strategic plans, structured, supplier management, supply chain, Supply Chain Management, switches, Switch, system integration,
Tableau, tax, user training, telecom, translating, upgrading, ventilation, Visio, VBA, workflow''']

In [17]:
for doc in nlp.pipe(main_str, disable=["tagger", "parser"]):
  for ent in doc.ents:
    print((ent.text,ent.label_))
    # print([(ent.text, ent.label_) for ent in doc.ents])

('MASTER DATA MANAGER', 'JOB TITLE')
('Master Data Manager\n\n,\n01/2019\n\nto\nCurrent', 'EXPERIENCE')
('Director of Information Technology\n\n,\n01/2017\n\nto\n01/2019', 'EXPERIENCE')
('Information Technology and Analytics Manager\n\n,\n01/2014\n\nto\n01/2017', 'EXPERIENCE')
('01/2015\n\nto\n01/2017', 'EXPERIENCE')
('Master Data Manager\n\n,\n01/2019\n\nto\nCurrent', 'EXPERIENCE')
('Director of Information Technology\n\n,\n01/2017\n\nto\n01/2019', 'EXPERIENCE')
('Information Technology and Analytics Manager\n\n,\n01/2014\n\nto\n01/2017', 'EXPERIENCE')
('Bachelor of Science\n\n:\nManagement of Information Systems', 'EDUCATION')
('Associates of Science\n\n:\nAccounting', 'EDUCATION')
('Rowan College', 'EDUCATION')
('Pursuing Project Management Professional', 'EDUCATION')
('Rowan University', 'EDUCATION')
('VBA', 'SKILL')
('DAX', 'SKILL')
('HTML', 'SKILL')
('CSS', 'SKILL')
('VB.NET', 'SKILL')
('R', 'SKILL')
('Powershell', 'SKILL')
('Python', 'SKILL')
('Oracle', 'SKILL')
('PowerBI', 'SKI